In [17]:
#### New
from functools import partial
from kfp.components import create_component_from_func, InputPath, OutputPath


@partial(
    create_component_from_func,
    packages_to_install=["pandas"],
)
def load_data(
    # : OutputPath("csv"),
    # evaluation_path: OutputPath("csv"),
):

    import pandas as pd
    import os

    print("list_dir : \n ", os.listdir())

    # load data from github
    df_train = pd.read_csv(
        "https://raw.github.com/yangoos57/Learning_kubeflow/main/mini_project/data/train.csv"
    )
    df_evaluation = pd.read_csv(
        "https://raw.github.com/yangoos57/Learning_kubeflow/main/mini_project/data/validation.csv"
    )

    df_train.to_csv("pvc/train.csv", index=False)
    df_evaluation.to_csv("pvc/evaluation.csv", index=False)

    print("complete Loading Data")


In [22]:
@partial(create_component_from_func, base_image="679oose/basepython:1.0")
def train_model(
    # train_path:InputPath("csv"),
    # evaluation_path: InputPath("csv"),
):

    from transformers import (
        DistilBertForSequenceClassification,
        DistilBertTokenizer,
        Trainer,
        TrainingArguments,
        TrainerCallback,
    )

    from datasets import Dataset

    import os

    print("list_dir : \n ", os.listdir())
    print("list_dir : \n ", os.getcwd())
    os.chdir("/")

    # loading data
    # train_dataset = Dataset.from_csv(train_path).select(range(100))
    # evaluation_dataset = Dataset.from_csv(evaluation_path)

    train_dataset = Dataset.from_csv("pvc/train.csv").select(range(32))
    evaluation_dataset = Dataset.from_csv("pvc/evaluation.csv")

    # tokenizing
    tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(item):
        return tokenizer(item["text"], padding="max_length", max_length=128, truncation=True)

    train = train_dataset.map(tokenize_function)
    evaluation = evaluation_dataset.map(tokenize_function)

    print("complete Tokenizing")

    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=len(set(train_dataset["label"]))
    )

    # BetterTransformer

    tra_arg = TrainingArguments(
        per_device_train_batch_size=8,
        output_dir="test",
        num_train_epochs=1,
        logging_steps=2,
        # evaluation_strategy="epoch",
        disable_tqdm=True,
        save_strategy="no",
    )

    class myCallback(TrainerCallback):
        def on_log(self, args, state, control, logs=None, **kwargs):
            print(f"{state.global_step} Steps ")

    trainer = Trainer(
        model=model,
        args=tra_arg,
        train_dataset=train,
        eval_dataset=evaluation,
        callbacks=[myCallback],
    )

    trainer.train()

    # Saving Tokenizer, Model
    trainer.save_model("pvc/torch_model")
    tokenizer.save_pretrained("pvc/torch_model")

    print("Saving Model & Tokenizer Complete !!")

    # config for torch
    config = dict(
        inference_address="http://0.0.0.0:8085",
        management_address="http://0.0.0.0:8085",
        metrics_address="http://0.0.0.0:8082",
        grpc_inference_port=7070,
        grpc_management_port=7071,
        enable_envvars_config="true",
        install_py_dep_per_model="true",
        model_store="model-store",
        model_snapshot={
            "name": "startup.cfg",
            "modelCount": 1,
            "models": {
                "torch-model": {  # Model Name
                    "1.0": {
                        "defaultVersion": "true",
                        "marName": "torch-model.mar",
                        "minWorkers": 1,
                        "maxWorkers": 5,
                        "batchSize": 1,
                        "maxBatchDelay": 500,
                        "responseTimeout": 120,
                    }
                }
            },
        },
    )
    
    # making config & config folder
    if not os.path.exists('pvc/torch_model/config') : 
        os.mkdir('pvc/torch_model/config')
        
    with open("pvc/torch_model/config/config.properties", "w") as f:
        for i, j in config.items():
            f.write(f"{i}={j}\n")
        f.close()

    print("Saving config.properties !!")

    # handler for torch
    x = '''
from abc import ABC
import json
import logging
import os

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from ts.torch_handler.base_handler import BaseHandler

logger = logging.getLogger(__name__)


class TransformersClassifierHandler(BaseHandler, ABC):
    def __init__(self):
        super(TransformersClassifierHandler, self).__init__()
        self.initialized = False

    def initialize(self, ctx):
        self.manifest = ctx.manifest

        properties = ctx.system_properties
        model_dir = properties.get("model_dir")
        self.device = torch.device(
            "cuda:" + str(properties.get("gpu_id")) if torch.cuda.is_available() else "cpu"
        )

        # Read model serialize/pt file
        self.model = AutoModelForSequenceClassification.from_pretrained(model_dir)
        self.tokenizer = AutoTokenizer.from_pretrained(model_dir)

        self.model.to(self.device)
        # BetterTransformer
        self.model.eval()

        logger.debug(f"Transformer model from path {model_dir} loaded successfully")

        # Read the mapping file, index to object name
        mapping_file_path = os.path.join(model_dir, "index_to_name.json")

        if os.path.isfile(mapping_file_path):
            with open(mapping_file_path) as f:
                self.mapping = json.load(f)
        else:
            logger.warning(
                "Missing the index_to_name.json file. Inference output will not include class name."
            )

        self.initialized = True

    def preprocess(self, data):
        """Very basic preprocessing code - only tokenizes.
        Extend with your own preprocessing steps as needed.
        """
        print("------- input data --------")
        print(data)
        text = data[0].get("data")
        if text is None:
            text = data[0].get("body")

        logger.info(f"Received text: {text}")

        inputs = self.tokenizer.encode_plus(text, add_special_tokens=True, return_tensors="pt")
        return inputs

    def inference(self, inputs):
        """
        Predict the class of a text using a trained transformer model.
        """
        # NOTE: This makes the assumption that your model expects text to be tokenized
        # with "input_ids" and "token_type_ids" - which is true for some popular transformer models, e.g. bert.
        # If your transformer model expects different tokenization, adapt this code to suit
        # its expected input format.
        inputs = inputs.to(self.device)

        prediction = self.model(**inputs)[0].argmax().item()
        logger.info(f"Model predicted: {prediction}")

        if self.mapping:
            prediction = self.mapping[str(prediction)]
        return [prediction]

    def postprocess(self, inference_output):
        # TODO: Add any needed post-processing of the model predictions here
        logger.info(f"Model Name: {self.model.config._name_or_path}")
        logger.info(f"Model predicted: {inference_output}")
        return inference_output


_service = TransformersClassifierHandler()


def handle(data, context):
    try:
        if not _service.initialized:
            _service.initialize(context)

        if data is None:
            return None

        data = _service.preprocess(data)
        data = _service.inference(data)
        data = _service.postprocess(data)

        return data
    except Exception as e:
        raise e


    '''
    with open("pvc/torch_model/handler.py", "w") as f:
        f.write(x)
    f.close()

    print("Saving handler.py complete !!")


In [23]:
from kfp.dsl import ContainerOp


def create_marfile():
    return ContainerOp(
        name="Creating Marfile",
        command=["/bin/sh"],
        image="python:3.9",
        arguments=[
            "-c",
            "cd pvc/torch_model; pip install torchserve torch-model-archiver torch-workflow-archiver; torch-model-archiver --model-name torch-model --version 1.0 --serialized-file pytorch_model.bin --handler handler.py --extra-files config.json,vocab.txt --force; mkdir model-store; mv -f torch-model.mar model-store"
        ],  # pip install => create mar file => make model_store folder => mv marfile to model_store
    )


In [31]:
@partial(
    create_component_from_func,
    packages_to_install=["kserve==0.8.0"],
)
def create_inference_model():

    from kserve import (
        constants,
        KServeClient,
        V1beta1InferenceService,
        V1beta1InferenceServiceSpec,
        V1beta1PredictorSpec,
        V1beta1ModelSpec,
        V1beta1ModelFormat,
        V1beta1TorchServeSpec,
        utils,
    )
    from kubernetes import client

    client.V1ResourceRequirements(requests={"cpu": 1, "memory": "1G"})

    service_name = "pytorchserve2"
    namespace = "kubeflow-user-example-com"
    api_version = constants.KSERVE_GROUP + "/" + constants.KSERVE_V1BETA1_VERSION
    storage = "pvc://lee/torch_model"

    torchsvc = V1beta1InferenceService(
        api_version=api_version,
        kind=constants.KSERVE_KIND,
        metadata=client.V1ObjectMeta(
            name=service_name, namespace=namespace, annotations={"sidecar.istio.io/inject": "false"}
        ),
        spec=V1beta1InferenceServiceSpec(
            predictor=V1beta1PredictorSpec(
                pytorch=(
                    V1beta1TorchServeSpec(
                        protocol_version="v1",
                        resources=client.V1ResourceRequirements(
                            requests={"cpu": 1, "memory": "1G"}
                        ),
                        storage_uri=storage,
                    )
                )
            )
        ),
    )
    KServe = KServeClient()
    KServe.create(torchsvc)


In [32]:
from kfp.dsl import pipeline
from kfp import onprem
from kfp.dsl import ContainerOp

@pipeline(name="NLP_Pipeline")
def NLP_Pipeline():
    data = load_data()
    data.apply(onprem.mount_pvc(pvc_name='lee', volume_name='test-lee', volume_mount_path="pvc"))
    # model = train_model(data.outputs['train'],data.outputs['evaluation'])    
    model = train_model()
    model.apply(onprem.mount_pvc(pvc_name='lee', volume_name='test-lee', volume_mount_path="pvc"))
    model.set_cpu_limit(cpu = '2').set_memory_limit(memory = '4G')
    model.set_display_name('Finetuning Text Classification Model')
    # model.execution_options.caching_strategy.max_cache_staleness = "P0D" # 매번 새롭게 시작하게 하는 명령어
    model.after(data)
    
    marfile=create_marfile()
    marfile.apply(onprem.mount_pvc(pvc_name='lee', volume_name='test-lee', volume_mount_path="pvc"))
    marfile.set_display_name('Creating Marfile')
    marfile.execution_options.caching_strategy.max_cache_staleness = "P0D" # 매번 새롭게 시작하게 하는 명령어
    marfile.after(model)

    inference_model=create_inference_model()
    inference_model.apply(onprem.mount_pvc(pvc_name='lee', volume_name='test-lee', volume_mount_path="pvc"))
    inference_model.after(marfile)

import kfp
if __name__ == "__main__":
    kfp.compiler.Compiler().compile(NLP_Pipeline, "NLP_Pipeline.yaml")

In [26]:
from kserve import (
    constants,
    KServeClient,
    V1beta1InferenceService,
    V1beta1InferenceServiceSpec,
    V1beta1PredictorSpec,
    V1beta1ModelSpec,
    V1beta1ModelFormat,
    V1beta1TorchServeSpec,
    utils,
)
from kubernetes import client

client.V1ResourceRequirements(requests={"cpu": 1, "memory": "1G"})

service_name = "pytorchserve"
namespace = "kubeflow-user-example-com"
api_version = constants.KSERVE_GROUP + "/" + constants.KSERVE_V1BETA1_VERSION
storage = "pvc://lee/torch_model"

torchsvc = V1beta1InferenceService(
    api_version=api_version,
    kind=constants.KSERVE_KIND,
    metadata=client.V1ObjectMeta(
        name=service_name, namespace=namespace, annotations={"sidecar.istio.io/inject": "false"}
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            pytorch=(
                V1beta1TorchServeSpec(
                    protocol_version="v1",
                    resources=client.V1ResourceRequirements(
                        requests={"cpu": 0.5, "memory": "0.5G"},
                        limits={"cpu": 1, "memory": "2G"}
                    ),
                    storage_uri=storage,
                )
            )
        )
    ),
)
KServe = KServeClient()
KServe.create(torchsvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2023-01-21T07:07:18Z',
  'generation': 1,
  'labels': {'serviceEnvelope': 'kserve'},
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {},
       'f:pytorch': {'.': {},
        'f:name': {},
        'f:protocolVersion': {},
        'f:resources': {'.': {},
         'f:limits': {'.': {}, 'f:cpu': {}, 'f:memory': {}},
         'f:requests': {'.': {}, 'f:cpu': {}, 'f:memory': {}}},
        'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2023-01-21T07:07:15Z'}],
  'name': 'pytorchserve',
  'namespace': 'kubeflow-user-example-com',
  'resourceVersion': '144461',
  'uid': 'a1699e07-c045